In [3]:
import pandas as pd

In [4]:
# Chargement du fichier CSV
df = pd.read_csv("jeu_donnees_etl_5000_lignes.csv")


In [5]:
# Affichage des 5 premières lignes
print(df.head())


   ID_produit Nom_produit  Quantite_vendue  Prix_unitaire  Date_vente
0           1     Chemise             10.0           25.0  2022-01-05
1           2    Pantalon              8.0           35.0  2022-01-06
2           3  Chaussures              NaN           50.0  2022-01-07
3           4     Cravate             12.0           15.0  2022-01-08
4           5        Robe             15.0           45.0  2022-01-09


In [6]:
# Suppression des doublons
df = df.drop_duplicates()


In [7]:
# Compter les valeurs manquantes
print(df.isnull().sum())


ID_produit            0
Nom_produit          10
Quantite_vendue    2476
Prix_unitaire      2364
Date_vente            0
dtype: int64


In [9]:
# Vérifier que toutes les dates sont valides
import datetime

def valider_date(chaine_date):
    try:
        datetime.datetime.strptime(chaine_date, "%Y-%m-%d")
        return True
    except ValueError:
        return False

# Liste des dates invalides
dates_invalides = df[~df["Date_vente"].apply(valider_date)]
print("Dates invalides :")
print(dates_invalides)


Dates invalides :
      ID_produit Nom_produit  Quantite_vendue  Prix_unitaire    Date_vente
65            66        Pull             20.0            NaN         2022-
66            82    Pantalon              NaN          32.04  invalid_date
69            98  Chaussures              NaN          35.01    07/09/2023
71            94      Montre              NaN            NaN  invalid_date
72            80        Pull             12.0          14.12    18/07/2024
...          ...         ...              ...            ...           ...
5060          61     Cravate              7.9          59.59    20/06/2023
5061          94      Montre              NaN            NaN    31/05/2024
5063          69     Cravate              NaN          77.66    14/08/2024
5064          19     Chemise             17.0            NaN    08/11/2024
5065          55    Pantalon              NaN          20.07  invalid_date

[3238 rows x 5 columns]


In [10]:
# Détection des valeurs négatives
quantites_neg = df[df["Quantite_vendue"] < 0]
prix_neg = df[df["Prix_unitaire"] < 0]

print("Quantités négatives :", len(quantites_neg))
print("Prix négatifs :", len(prix_neg))


Quantités négatives : 0
Prix négatifs : 0


In [ ]:
# On supprime les lignes avec des quantités ou prix négatifs
# df = df[df["Quantite_vendue"] >= 0]
# df = df[df["Prix_unitaire"] >= 0]

#Il n'y a pas de quantités ou de prix négatifs

In [12]:
# Création d'une colonne "Total_vente"
df["Total_vente"] = df["Quantite_vendue"] * df["Prix_unitaire"]


In [13]:
# Création d'une colonne "TVA" (20 % de la vente)
df["TVA"] = df["Total_vente"] * 0.20


In [ ]:
from datetime import datetime

# Ajout de la colonne "Date_traitement" avec la date actuelle
df["Date_traitement"] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
print(df.head())

   ID_produit Nom_produit  Quantite_vendue  Prix_unitaire  Date_vente  \
0           1     Chemise             10.0           25.0  2022-01-05   
1           2    Pantalon              8.0           35.0  2022-01-06   
3           4     Cravate             12.0           15.0  2022-01-08   
4           5        Robe             15.0           45.0  2022-01-09   
6           7    Pantalon              8.0           35.0  2022-01-06   

   Total_vente    TVA      Date_traitement  
0        250.0   50.0  2025-05-22 12:16:22  
1        280.0   56.0  2025-05-22 12:16:22  
3        180.0   36.0  2025-05-22 12:16:22  
4        675.0  135.0  2025-05-22 12:16:22  
6        280.0   56.0  2025-05-22 12:16:22  


In [16]:
# Normalisation Min-Max de Total_vente
total_min = df["Total_vente"].min()
total_max = df["Total_vente"].max()

df["Total_vente_normalise"] = (df["Total_vente"] - total_min) / (total_max - total_min)


In [17]:
def classer_produit(prix):
    if prix > 40:
        return "Luxe"
    else:
        return "Basique"

df["Categorie_produit"] = df["Prix_unitaire"].apply(classer_produit)


In [18]:
import numpy as np

# Liste pour journaliser les erreurs
log_erreurs = []

# Fonction avec gestion d'erreurs
def calculer_ratio(row):
    try:
        return row["Total_vente"] / row["Quantite_vendue"]
    except ZeroDivisionError:
        log_erreurs.append(f"Division par 0 pour ID {row['ID_produit']}")
        return np.nan
    except Exception as e:
        log_erreurs.append(f"Erreur {type(e).__name__} pour ID {row['ID_produit']} : {e}")
        return np.nan

# Application
df["Ratio_vente_quantite"] = df.apply(calculer_ratio, axis=1)


In [19]:
df[["ID_produit", "Quantite_vendue", "Total_vente", "Ratio_vente_quantite"]].head()


,ID_produit,Quantite_vendue,Total_vente,Ratio_vente_quantite
0,1,10.0,250.0,25.0
1,2,8.0,280.0,35.0
3,4,12.0,180.0,15.0
4,5,15.0,675.0,45.0
6,7,8.0,280.0,35.0


In [20]:
# Sauvegarde de la dernière version transformée (remplacée à chaque exécution)
df.to_csv("donnees_traitees.csv", index=False)


In [22]:
import os
from datetime import datetime

# Crée le dossier 'historique' s'il n'existe pas
os.makedirs("historique", exist_ok=True)

# Générer un nom de fichier horodaté
timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
nom_fichier = f"donnees_traitees_{timestamp}.csv"

# Chemin complet dans le dossier 'historique'
chemin_complet = os.path.join("historique", nom_fichier)

# Sauvegarder dans le dossier historique
df.to_csv(chemin_complet, index=False)

print(f"Sauvegarde horodatée créée dans : {chemin_complet}")


Sauvegarde horodatée créée dans : historique\donnees_traitees_2025-05-22_15-34-17.csv
